In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from IPython import embed
import pickle
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch.nn.functional as F #functions
from torchvision import datasets, transforms
import sklearn.metrics as sk_m
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import os
import shutil

import sys
sys.path.append(os.path.abspath("/Users/andyvarner/Documents/NN_Spring2023/project_2"))
import dataset_methods

%matplotlib notebook

In [2]:
class ChessRNN(torch.nn.Module): #inherits from nn.Module
    
    #def __init__(self, input_size, hidden_size, num_layers, lr, output_size):
    def __init__(self, input_size, output_size, hidden_dim, num_layers, lr, nonlinearity='relu'):
        # hidden_dim = the number of features in the hidden state h
        # num_layers = Number of recurrent layers. 
        #              E.g., setting num_layers=2 would mean stacking two RNNs together to form a stacked 
        #              RNN, with the second RNN taking in outputs of the first RNN and computing the final 
        #              results. Default: 1
        # nonlinearity = The non-linearity to use. Can be either 'tanh' or 'relu'. Default: 'tanh'
        
        super(ChessRNN, self).__init__() #initialize nn.Module
        
        self.rnn = nn.RNN(input_size, hidden_dim, num_layers=num_layers, batch_first=True,)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lr = lr
        
    def forward(self, x, num_turns):
        
        embed();exit()
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        out = out[0][num_turns-1]
        
        return out
        
    def init_optimizer(self):
        
        self.optimizer = torch.optim.Adam(self.parameters(), lr = self.lr)

#     def objective(self, outputs, labels): # this is the loss function

#         loss = torch.nn.CrossEntropyLoss(outputs, labels)
        
#         return loss
    
    def init_hidden(self, batch_size):
        
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim)
    
def train_rnn(model, train_dataset, test_dataset, num_epochs = 10, rate = 2):
    
    criterion = nn.CrossEntropyLoss()
    
    training_losses, valid_losses, train_labels, train_preds, cf_matrices = [], [], [], [], []

    model.init_optimizer()
   
    for epoch in range(num_epochs):
        train_epoch_loss, train_epoch_acc, valid_epoch_loss, valid_epoch_acc = 0, 0, 0, 0

        model.train()        # Set model to train mode

        # Loop over the train data.
        for i, (turns, winner, num_turns), in enumerate(tqdm(train_dataset, desc = "Train Epoch %s" % epoch)): ### turns = inputs, winner = labels

            # Zero out gradients #
            model.optimizer.zero_grad()
            
            # Forward pass #
            outputs = model(turns, num_turns)
            
            #loss = criterion(outputs, winner.float())
            loss = criterion(outputs, torch.argmax(winner, dim=1))  # is this actually correct?
            
            # Backward pass #
            loss.backward()
            model.optimizer.step()
            
            # update training loss #
            train_epoch_loss += loss.item()
            
            # Update training accuracy
            train_pred = F.softmax(outputs, dim=1)
            _, train_pred_labels = torch.max(train_pred, dim=1)
            train_epoch_acc += torch.sum(train_pred_labels == torch.argmax(winner, dim=1)).item() 
            
#             train_preds.append(winner.numpy())
#             train_labels.append(winner)


        train_loss = train_epoch_loss / len(train_dataset)
        train_acc = train_epoch_acc / len(train_dataset)
        training_losses.append(train_loss)
        train_labels.append(winner)
        train_preds.append(train_pred_labels)

        
        ###############
        
        print(f"epoch {epoch}, epoch_loss={train_loss}, epoch_acc={train_acc}")

        # Validate network
        
        if(epoch % rate == 0):
            print("test")
            
            model.eval()
            
            valid_epoch_loss, valid_epoch_acc = 0, 0
            valid_labels, valid_preds = [], []

            for i, (turns, winner, num_turns), in enumerate(tqdm(test_dataset, desc = "Test Epoch %s" % epoch)):   
                
                outputs = model(turns, num_turns)
                
                # Update Loss
                loss = criterion(outputs, torch.argmax(winner, dim=1))
                valid_epoch_loss += loss.item()
                
                # Update validation accuracy
                preds = F.softmax(outputs, dim=1)          # the model's prediction
                _, pred_labels = torch.max(preds, dim=1)   
                valid_epoch_acc += torch.sum(pred_labels == torch.argmax(winner, dim=1)).item()
                
                valid_labels.append(int(torch.argmax(winner)))
                valid_preds.append(pred_labels.item())
                #valid_preds.append(int(torch.argmax(preds)))

            valid_losses.append(valid_epoch_loss / (len(test_dataset) + 1e-6))
            valid_acc = valid_epoch_acc / (len(test_dataset) + 1e-6)

            cf_matrix = sk_m.confusion_matrix(np.asarray(valid_labels), np.asarray(valid_preds))
            cf_matrices.append(cf_matrix)

            print("Valid Accuracy %s Valid Loss %s" % (valid_acc, valid_losses))
                
            model.train()
    
        ##get metrics
            
        metrics = {}
        metrics["valid_losses"] = valid_losses
        metrics["training_losses"] = training_losses
        metrics["labels"] = valid_labels
        metrics["preds"] = valid_preds
        metrics["mats"] = cf_matrices

    #return training_loss, training_metrics
    return metrics

In [ ]:
# Set hyperparameters - Don't change these
input_size = 5    # number of features in the input (shape of `moves` encoding)
output_size = 2   # number of output classes ([white, black], 0-loser, 1-winner)

# hidden_dim = the number of features in the hidden state h
# num_layers = Number of recurrent layers. 
#              e.g., setting num_layers=2 would mean stacking two RNNs together to form a stacked 
#              RNN, with the second RNN taking in outputs of the first RNN and computing the final 
#              results. Default: 1
# nonlinearity = The non-linearity to use. Can be either 'tanh' or 'relu'. Default: 'tanh'

# These can change
hidden_dim =  32 # number of hidden units in the RNN
rate = 2 # how often we run the validation loop
num_layers = 4  # number of recurrent layers

learning_rate = 0.001
num_epochs = 10

model = ChessRNN(input_size, output_size, hidden_dim, num_layers, learning_rate)

filename = "games_10.pkl"   # the integer value is the number of moves we're looking at

train, test = dataset_methods.get_dataset(filename)
# embed()

train_metrics = train_rnn(model, train, test, num_epochs = num_epochs, rate = rate)

Train Epoch 0:   0%|                                                                                                   | 0/252 [00:00<?, ?it/s]

Python 3.9.16 | packaged by conda-forge | (main, Feb  1 2023, 21:38:11) 
Type 'copyright', 'credits' or 'license' for more information
IPython 8.11.0 -- An enhanced Interactive Python. Type '?' for help.



In [1]:  x.shape


Out[1]: torch.Size([1, 100, 5])



In [2]:  x[0].shape


Out[2]: torch.Size([100, 5])



In [3]:  x[1].shape


---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[3], line 1
----> 1 x[1].shape

IndexError: index 1 is out of bounds for dimension 0 with size 1



In [4]:  x[-1].shape


Out[4]: torch.Size([100, 5])



In [17]:
path_save = f"/Users/andyvarner/Documents/NN_Spring2023/project_2/Results5/epochs_{num_epochs}"

if not os.path.exists(path_save):
    os.makedirs(path_save)
    print(f"Folder created at {path_save}")
else:
    print(f"Folder already exists at {path_save}")

title = "%s" % (str(filename).zfill(3))
file_save = os.path.join(path_save, title)

pickle.dump(train_metrics, open(file_save, "wb"))

Folder already exists at /Users/andyvarner/Documents/NN_Spring2023/project_2/Results5/epochs_10
